In [1]:
import environ
import gspread
from sqlalchemy import create_engine
import json


env = environ.Env()

database_url = env.str(
    var="DATABASE_URL",
    default="postgres://postgres:@postgres:5432/postgres"
)
aas=['id', 'company_id', 'user_id', 'done', 'type','reference_type', 'reference_id', 
                 'conference_meeting_client', 'conference_meeting_url', 'due_date', 'due_time', 'duration', 
                 'busy_flag','add_time','marked_as_done_time','last_notification_time',
                 'last_notification_user_id','notification_language_id','subject','public_description',
                'calendar_sync_include_context','location','org_id','person_id','deal_id','lead_id','lead_title',
                'active_flag','update_time','update_user_id','gcal_event_id','google_calendar_id',
                 'google_calendar_etag','source_timezone','rec_rule','rec_rule_extension','rec_master_activity_id',
                'conference_meeting_id','note','created_by_user_id','location_subpremise','location_street_number',
                 'location_route','location_sublocality','location_locality','location_admin_area_level_1',
                 'location_admin_area_level_2','location_country','location_postal_code',
                'location_formatted_address','series','org_name','person_name','deal_title','owner_name',
                     'person_dropbox_bcc','deal_dropbox_bcc','assigned_to_user_id','type_name','file']


In [2]:
# from pipedrive import Pipedrive
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from pipedrive.client import Client


gc = gspread.service_account(filename="./credentials.json")
engine = create_engine(database_url, echo=False)

In [3]:
# pipedrive = Pipedrive('c70f38803bb436089bce792681cd178e8459c86e')
# ppd = pipedrive.activities(method='GET')

In [4]:
client = Client(domain='https://companydomain.pipedrive.com/')
client.set_api_token('c70f38803bb436089bce792681cd178e8459c86e')
act_data = client.activities.get_all_activities(params={"limit":"100000"})
act_data = pd.DataFrame(act_data['data'])
# act_data['attendees'] = list(map(lambda x: json.dumps(x), act_data['attendees']))
act_data = act_data[aas]
# act_data = act_data.drop(['attendees'], axis=1)

# print(act_data)

# act_data.to_sql('etl_sale_activities', engine, if_exists='replace', index=False)

In [5]:
# response['additional_data']

In [6]:
url = 'https://docs.google.com/spreadsheets/d/1ab4eW4deYLV-3mMgAMuafoWM3e7JjYzoLDAJ83WPuaA/edit#gid=13444138'
workbook = gc.open_by_url(url)
# sheet = workbook.get_worksheet(3)
# set_with_dataframe(sheet, act_data1)

In [7]:
org = client.organizations.get_all_organizations()
actf = org['data']
actf = pd.DataFrame(actf)
# sheetorg = workbook.get_worksheet(4)
# set_with_dataframe(sheetorg, actf)
# actf.head()

In [9]:
deals = client.deals.get_all_deals(params={"limit":"100000"})
ddata = deals['data']
ddf = pd.DataFrame(ddata)
ddf
sheetdeals = workbook.get_worksheet(5)
# set_with_dataframe(sheetdeals, ddf)
# ddf.head()

In [10]:
t1 = list(ddf['id'])

In [11]:
stage = client.stages.get_all_stages()
dstage = stage['data']
sdf = pd.DataFrame(dstage)
sdf
sheetstage = workbook.get_worksheet(3)
set_with_dataframe(sheetstage, sdf, row=50)
# sdf.head()
# sdf.to_sql('etl_sale_stages', engine, if_exists='replace', index=False)

In [12]:
st_deal = client.stages.get_stage_deals(62)
st_deal = pd.DataFrame(st_deal['data'])
# st_deal


In [13]:
se_deal = client.stages.get_stage_deals(2)
se_deal = pd.DataFrame(se_deal['data'])
# se_deal

In [14]:
product = client.products.get_all_products()
product = pd.DataFrame(product['data'])

# sheetstage = workbook.get_worksheet(7)
# set_with_dataframe(sheetstage, product)
producr_id = list(product['id'])
producr_id

[2, 3, 4, 5, 6, 7, 8, 9, 10]

In [15]:
# product.head()

In [16]:
product_list = list()
for i in producr_id:
    pdp = client.products.get_product_deal(str(i))
    if pdp['data']:
        for j in pdp['data']:
            product_list.append({'product_id': i, 'deal_id': j['id']})
product_list = pd.DataFrame(product_list)
product_list

,product_id,deal_id
0,2,94
1,2,64
2,2,25
3,3,94
4,3,64
5,3,25
6,4,94
7,7,243
8,8,64
9,9,243


In [17]:
ddf_re = ddf.rename(columns={'id': 'deal_id'})
product_re = product.rename(columns={'id': 'product_id'})
product_df = product_list.join(product_re.set_index('product_id'), on='product_id')
product_df[['product_id', 'name']]
product_df = product_df.rename(columns={'name':'product_name'})

In [18]:
product_df = product_df[['product_id', 'product_name', 'deal_id']]
product_df

,product_id,product_name,deal_id
0,2,e-Memo Set Up,94
1,2,e-Memo Set Up,64
2,2,e-Memo Set Up,25
3,3,Subscribe Monthly Fee,94
4,3,Subscribe Monthly Fee,64
5,3,Subscribe Monthly Fee,25
6,4,Installation Fee(Sub company),94
7,7,E-Memo,243
8,8,Additional Request,64
9,9,e-Tax,243


In [19]:
col = ['deal_id','creator_user_id','user_id', 'person_id','org_id','stage_id',
           'title','value', 'currency', 'add_time', 'update_time','stage_change_time', 'active', 'deleted', 'status']

In [20]:
# product_df = get_pipedrive_product()
product_df = product_df[['product_id', 'name', 'deal_id']]
deals_data = client.deals.get_all_deals()
deals_data = pd.DataFrame(deals_data['data'])


deals_data['creator_user_id'] = list(map(lambda x: json.dumps(x), deals_data['creator_user_id']))
deals_data['person_id'] = list(map(lambda x: json.dumps(x), deals_data['person_id']))
deals_data['user_id'] = list(map(lambda x: json.dumps(x), deals_data['user_id']))
deals_data['org_id'] = list(map(lambda x: json.dumps(x), deals_data['org_id']))


ddf_re = deals_data.rename(columns={'id': 'deal_id'})
ddf_join = ddf_re.join(product_df.set_index('deal_id'), on='deal_id')

# print(ddf_join)

# ddf_join.to_sql('etl_sale_deals', engine, if_exists='replace', index=False)

KeyError: "['name'] not in index"

In [22]:
get_deal = client.deals.get_deal(deal_id=100)
get_deals

NameError: name 'get_deals' is not defined

In [26]:
dtest1 = list(ddf['id'])
get_deals_list = list()
for i in dtest1:
    deal_de = client.deals.get_deal(deal_id=int(i))
    get_deals_list.append(deal_de["data"])
gd_pd = pd.DataFrame(get_deals_list)
gd_pd
# sheetstage = workbook.get_worksheet(9)
# set_with_dataframe(sheetstage, gd_pd)

,id,creator_user_id,user_id,person_id,org_id,stage_id,title,value,currency,add_time,...,owner_name,cc_email,org_hidden,person_hidden,average_time_to_won,average_stage_progress,age,stay_in_pipeline_stages,last_activity,next_activity
0,10,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'ยงยุทธ', 'email...","{'name': 'บริษัท วี.เค.จี กรุ๊ป จำกัด', 'peopl...",62,บริษัท วี.เค.จี กรุ๊ป จำกัด,0,THB,2020-07-13 03:35:33,...,Sales Tracking,codium2+deal10@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 0, 'h': 0, 'i': 0, 's': ...",0.00,"{'y': 0, 'm': 7, 'd': 12, 'h': 0, 'i': 8, 's':...","{'times_in_stages': {'1': 14113011, '62': 5500...",None,None
1,11,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'ทวี สกุลคุณสวัส...",None,2,ทวี สกุลคุณสวัสดิ์,0,THB,2020-07-13 03:42:29,...,Sales Tracking,codium2+deal11@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 0, 'h': 0, 'i': 0, 's': ...",0.00,"{'y': 0, 'm': 0, 'd': 11, 'h': 3, 'i': 33, 's'...","{'times_in_stages': {'1': 963200, '2': 0}, 'or...",None,None
2,12,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'ทิศากร ยูวะเวศ'...","{'name': 'กรมยุทธศึกษาทหารบก', 'people_count':...",62,กรมยุทธศึกษาทหารบก,0,THB,2020-07-13 03:45:53,...,Sales Tracking,codium2+deal12@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 0, 'h': 0, 'i': 0, 's': ...",0.00,"{'y': 0, 'm': 7, 'd': 11, 'h': 23, 'i': 58, 's...","{'times_in_stages': {'1': 14112390, '62': 5500...","{'id': 139, 'company_id': 7617199, 'user_id': ...",None
3,13,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'สุรัตน์', 'emai...","{'name': 'บริษัท อาบาคัสซึ', 'people_count': 1...",62,บริษัท อาบาคัสซึ,0,THB,2020-07-13 03:53:13,...,Sales Tracking,codium2+deal13@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 0, 'h': 0, 'i': 0, 's': ...",0.00,"{'y': 0, 'm': 7, 'd': 11, 'h': 23, 'i': 50, 's...","{'times_in_stages': {'1': 14111949, '62': 5500...","{'id': 140, 'company_id': 7617199, 'user_id': ...",None
4,14,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'Warathep Chalad...","{'name': 'SNC FORMER PUBLIC COMPANY LIMITED.',...",62,SNC FORMER PUBLIC COMPANY LIMITED.,0,THB,2020-07-13 04:07:23,...,Sales Tracking,codium2+deal14@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 0, 'h': 0, 'i': 0, 's': ...",0.00,"{'y': 0, 'm': 7, 'd': 11, 'h': 23, 'i': 36, 's...","{'times_in_stages': {'1': 14111070, '62': 5500...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,443,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'เนรมิต เลา', 'e...","{'name': 'บริษัท พีพี แกลม จำกัด', 'people_cou...",55,บริษัท พีพี แกลม จำกัด Far,0,THB,2021-02-24 03:26:50,...,Sales Tracking,codium2+deal443@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 22, 'h': 2, 'i': 47, 's'...",2.15,"{'y': 0, 'm': 0, 'd': 1, 'h': 0, 'i': 20, 's':...","{'times_in_stages': {'55': 87654}, 'order_of_s...",None,None
383,444,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'สุจิตรา เกษสุวร...","{'name': 'KE Group', 'people_count': 1, 'owner...",56,KE Group - Toon,0,THB,2021-02-24 03:52:05,...,Sales Tracking,codium2+deal444@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 22, 'h': 2, 'i': 47, 's'...",2.14,"{'y': 0, 'm': 0, 'd': 0, 'h': 23, 'i': 55, 's'...","{'times_in_stages': {'55': 2100, '56': 84038, ...",None,None
384,445,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag

In [ ]:
# sheetstage = workbook.get_worksheet(7)
# set_with_dataframe(sheetstage, gd_pd, row=15)

In [28]:
time_stay_stage = gd_pd[['stay_in_pipeline_stages', 'id', 'pipeline_id', 'status']]
meeting_time = None
inprocess_time = None
won_or_lost_time = None
lead_in = None
Contact  = None
Meeting = None
Negotiations_Started = None
wfc = None
In_process = None
won_or_lost = None
# sheetstage = workbook.get_worksheet(4)
# set_with_dataframe(sheetstage, time_stay_stage, row=120)
time_stay_stage

,stay_in_pipeline_stages,id,pipeline_id,status
0,"{'times_in_stages': {'1': 14113011, '62': 5500...",10,1,open
1,"{'times_in_stages': {'1': 963200, '2': 0}, 'or...",11,1,lost
2,"{'times_in_stages': {'1': 14112390, '62': 5500...",12,1,open
3,"{'times_in_stages': {'1': 14111949, '62': 5500...",13,1,open
4,"{'times_in_stages': {'1': 14111070, '62': 5500...",14,1,open
...,...,...,...,...
382,"{'times_in_stages': {'55': 87654}, 'order_of_s...",443,10,open
383,"{'times_in_stages': {'55': 2100, '56': 84038, ...",444,10,open
384,"{'times_in_stages': {'55': 5, '56': 85036, '61...",445,10,open
385,"{'times_in_stages': {'55': 2571, '61': 70437},...",446,10,open


In [29]:
# time_dict = {}
time_list = []
for idx, row in time_stay_stage.iterrows():
    time_dict = {}
    meeting_time = None
    inprocess_time = None
    won_or_lost_time = None
    time_range = row['stay_in_pipeline_stages']['times_in_stages']
    if row['status'] in ['won', 'lost']:
        if row['pipeline_id'] == 1:
            if '2' in time_range.keys():
                meeting_time = time_range['2']
            if '35' in time_range.keys():
                inprocess_time = time_range['35']
            if '3' in time_range.keys() and '6' in time_range.keys():
                won_or_lost_time = time_range['3'] if row['status'] == 'won' else time_range['6']

        elif row['pipeline_id'] == 5:
            if '25' in time_range.keys():
                meeting_time = time_range['25']
            if '65' in time_range.keys():
                inprocess_time = time_range['65']
            if '28' in time_range.keys():
                won_or_lost_time = time_range['28'] if row['status'] == 'won' else None
        else:
            continue

        time_dict['meeting_time'] = meeting_time if meeting_time else None
        time_dict['inprocess_time'] = inprocess_time if inprocess_time else None
        time_dict['won_or_lost_time'] = won_or_lost_time if won_or_lost_time else None
        time_dict['id'] = row['id']
        print(time_dict)
        time_list.append(time_dict)
    
print(time_list)

#         print(row['id'], row['stay_in_pipeline_stages'], row['pipeline_id'], row['status'])

{'meeting_time': None, 'inprocess_time': None, 'won_or_lost_time': None, 'id': 11}
{'meeting_time': None, 'inprocess_time': 4061988, 'won_or_lost_time': None, 'id': 25}
{'meeting_time': None, 'inprocess_time': None, 'won_or_lost_time': None, 'id': 28}
{'meeting_time': None, 'inprocess_time': 166303, 'won_or_lost_time': None, 'id': 29}
{'meeting_time': 2148066, 'inprocess_time': None, 'won_or_lost_time': None, 'id': 30}
{'meeting_time': None, 'inprocess_time': 334372, 'won_or_lost_time': None, 'id': 38}
{'meeting_time': None, 'inprocess_time': 2146707, 'won_or_lost_time': None, 'id': 40}
{'meeting_time': None, 'inprocess_time': None, 'won_or_lost_time': None, 'id': 44}
{'meeting_time': 54, 'inprocess_time': None, 'won_or_lost_time': None, 'id': 45}
{'meeting_time': None, 'inprocess_time': None, 'won_or_lost_time': None, 'id': 46}
{'meeting_time': None, 'inprocess_time': None, 'won_or_lost_time': None, 'id': 47}
{'meeting_time': None, 'inprocess_time': None, 'won_or_lost_time': None, 'id

In [30]:
time_df = pd.DataFrame(time_list)
time_df
tts = time_stay_stage.merge(time_df, 'inner', on='id')
# tts.drop(columns=[])
# sheetstage = workbook.get_worksheet(8)
# set_with_dataframe(sheetstage, tts)
tts

,stay_in_pipeline_stages,id,pipeline_id,status,meeting_time,inprocess_time,won_or_lost_time
0,"{'times_in_stages': {'1': 963200, '2': 0}, 'or...",11,1,lost,NaN,NaN,None
1,"{'times_in_stages': {'1': 341107, '2': 0, '4':...",25,1,won,NaN,4061988.0,None
2,"{'times_in_stages': {'1': 0, '2': 0, '5': 6644...",28,1,lost,NaN,NaN,None
3,"{'times_in_stages': {'1': 0, '2': 0, '4': 1646...",29,1,won,NaN,166303.0,None
4,"{'times_in_stages': {'24': 34, '25': 2148066, ...",30,5,won,2148066.0,NaN,None
5,"{'times_in_stages': {'1': 0, '2': 0, '4': 0, '...",38,1,won,NaN,334372.0,None
6,"{'times_in_stages': {'1': 0, '2': 0, '4': 0, '...",40,1,won,NaN,2146707.0,None
7,"{'times_in_stages': {'1': 0, '2': 0, '4': 26, ...",44,1,lost,NaN,NaN,None
8,"{'times_in_stages': {'1': 0, '2': 54}, 'order_...",45,1,lost,54.0,NaN,None
9,"{'times_in_stages': {'1': 46}, 'order_of_stage...",46,1,lost,NaN,NaN,None


In [31]:
time_list2 = []
sum_list = []
stage_pipeline = {1: [1, 62, 2, 5, 4, 35, 3, 6], 5: [24, 64, 25, 26, 27, 65, 28], 
                  10: [55, 61, 56, 57, 58, 59, 60]}
for idx, row in time_stay_stage.iterrows():
    time_dict = {}
    time_sum = 0
    sum_time_list = []
    time_range = row['stay_in_pipeline_stages']['times_in_stages']
    if row['status'] in ['won', 'lost']:
        if row['pipeline_id'] == 1:
            for ind in stage_pipeline[1]:
                sing = True
                if str(ind) in time_range.keys():
                    time_sum += time_range[str(ind)]
                else:
                    sing = False
                sum_time_list.append(time_sum)
            time_dict['lead_to_meeting'] = sum_time_list[2]
            time_dict['meeting_to_win'] = sum_time_list[6]
            time_dict['meeting_to_lost'] = sum_time_list[7]
            time_dict['id'] = row['id']
            time_list2.append(time_dict)
            
        if row['pipeline_id'] == 5:
            for ind in stage_pipeline[5]:
                sing = True
                if str(ind) in time_range.keys():
                    time_sum += time_range[str(ind)]
                else:
                    sing = False
                sum_time_list.append(time_sum)
            time_dict['lead_to_meeting'] = sum_time_list[2]
            time_dict['meeting_to_win'] = sum_time_list[6]
            time_dict['meeting_to_lost'] = None
            time_dict['id'] = row['id']
            time_list2.append(time_dict)
        
        if row['pipeline_id'] == 10:
            for ind in stage_pipeline[10]:
                sing = True
                if str(ind) in time_range.keys():
                    time_sum += time_range[str(ind)]
                else:
                    sing = False
                sum_time_list.append(time_sum)
            time_dict['lead_to_meeting'] = sum_time_list[2]
            time_dict['meeting_to_win'] = sum_time_list[6]
            time_dict['meeting_to_lost'] = None
            time_dict['id'] = row['id']
            time_list2.append(time_dict)

In [32]:
tl_df2 = pd.DataFrame(time_list2)
tll = time_stay_stage.merge(tl_df2, 'inner', on='id')
tll = tll.rename(columns={'id': 'deal_id'})


tll = tll.drop(columns=['stay_in_pipeline_stages'])

# sheetstage = workbook.get_worksheet(8)
# set_with_dataframe(sheetstage, tll, row=40)

# tll.to_sql("etl_sale_deals_time_of_stage", engine, if_exists="replace", index=False)
# tll


In [33]:
def time_range_stage(data):
    time_list2 = []
    stage_pipeline = {1: [1, 62, 2, 69, 5, 4, 35], 5: [24, 64, 25, 68, 26, 27, 65],
                      10: [55, 61, 56, 67, 57, 58, 59]}
    for idx, row in data.iterrows():
        time_dict = {}
        time_sum = 0
        sum_time_list = []
        time_range = row['stay_in_pipeline_stages']['times_in_stages']
        if row['status'] in ['won', 'lost']:
            if row['pipeline_id'] == 1:
                time_list2.append(cal_time(time_range, row, stage_pipeline[1]))

            elif row['pipeline_id'] == 5:
                time_list2.append(cal_time(time_range, row, stage_pipeline[5]))

            elif row['pipeline_id'] == 10:
                time_list2.append(cal_time(time_range, row, stage_pipeline[10]))

    tl_df = pd.DataFrame(time_list2)

    return tl_df

In [34]:
def cal_time(time_range, row, stage):
    time_dict = {}
    time_sum = 0
    sum_time_list = []
    
    for ind in stage:
        if str(ind) in time_range.keys():
            time_sum += time_range[str(ind)]
        sum_time_list.append(time_sum)
    time_dict['lead_to_meeting'] = sum_time_list[3]
    time_dict['meeting_to_Final'] = sum_time_list[6]
    time_dict['id'] = row['id']

    return time_dict

In [35]:
dsa = time_range_stage(time_stay_stage)
sheetstage = workbook.get_worksheet(4)
set_with_dataframe(sheetstage, dsa)
dsa

,lead_to_meeting,meeting_to_Final,id
0,963200,963200,11
1,341107,4403095,25
2,0,6644327,28
3,0,2148487,29
4,2148100,2148100,30
...,...,...,...
103,2407595,2407595,344
104,603603,1910874,346
105,2333140,2333140,347
106,272381,272381,353


In [37]:
note = client.notes.get_all_notes(params={"limit":"100000"})

In [40]:
df_note = pd.DataFrame(note['data'])
df_note

,id,user_id,deal_id,person_id,org_id,lead_id,content,add_time,update_time,active_flag,pinned_to_deal_flag,pinned_to_person_flag,pinned_to_organization_flag,pinned_to_lead_flag,last_update_user_id,organization,person,deal,user
0,1,11611129,1.0,1.0,1.0,None,Created by Web Forms using following informati...,2020-07-09 07:41:45,2020-07-09 07:41:45,True,False,False,False,False,NaN,{'name': 'กรมยุทธศึกษาทหารบก'},{'name': 'ทิศากร ยูวะเวส'},{'title': '[CLHM_WF] กรมยุทธศึกษาทหารบก deal'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
1,2,11611129,3.0,3.0,3.0,None,Created by Web Forms using following informati...,2020-07-09 09:21:44,2020-07-09 09:21:44,True,False,False,False,False,NaN,{'name': 'บริษัท บางกอกกล๊าซ จำกัด'},{'name': 'พิมุก'},{'title': '[CLHM_WF] บริษัท บางกอกกล๊าซ จำกัด ...,"{'email': 'presale@codium.co', 'name': 'Sales ..."
2,3,11611129,4.0,4.0,4.0,None,Created by Web Forms using following informati...,2020-07-09 09:24:19,2020-07-09 09:24:19,True,False,False,False,False,NaN,{'name': 'บริษัท คอบร้า อินเตอร์เนชั่นแนล จำกัด'},{'name': 'Prasit Sermsiripoka'},{'title': '[CLHM_WF] บริษัท คอบร้า อินเตอร์เนช...,"{'email': 'presale@codium.co', 'name': 'Sales ..."
3,4,11611279,1.0,1.0,1.0,None,ต้องการทราบรายละเอียด&nbsp; <div>1.จำนวนหน่วยง...,2020-07-09 09:33:46,2020-07-09 09:33:46,True,False,False,False,False,NaN,{'name': 'กรมยุทธศึกษาทหารบก'},{'name': 'ทิศากร ยูวะเวส'},{'title': '[CLHM_WF] กรมยุทธศึกษาทหารบก deal'},"{'email': 'tip.khattiyaporn@gmail.com', 'name'..."
4,5,11611279,1.0,1.0,1.0,None,Call 22.06.20 โรหาลูกค้า ลูกค้าขอเวลาลองศึกษาก...,2020-07-09 09:35:30,2020-07-09 09:35:30,True,False,False,False,False,NaN,{'name': 'กรมยุทธศึกษาทหารบก'},{'name': 'ทิศากร ยูวะเวส'},{'title': '[CLHM_WF] กรมยุทธศึกษาทหารบก deal'},"{'email': 'tip.khattiyaporn@gmail.com', 'name'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,501,11611129,278.0,255.0,222.0,None,"<p style=""margin-top:0pt;margin-bottom:0pt;"">D...",2021-01-11 07:19:10,2021-01-11 07:19:10,True,False,False,False,False,NaN,{'name': 'บ. พีรพัฒน์'},{'name': 'สิทธิโชค และ คุณภูมิ'},{'title': 'Peerapat - Toon'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
496,502,11611129,279.0,256.0,223.0,None,สนใจตัวระบบการจัดการเอกสารและการอนุมัติ เป็นพน...,2021-01-11 07:21:24,2021-01-11 07:21:24,True,False,False,False,False,NaN,{'name': 'ลิฟวิ่ง - Living Deal - Jeff'},{'name': 'นวพร'},{'title': 'ลิฟวิ่ง Living deal - Jeff'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
497,503,11611129,280.0,257.0,224.0,None,ต้องการข้อมูลของระบบ อยากได้การทำงานด้านเอกสาร...,2021-01-11 07:22:37,2021-01-11 07:22:37,True,False,False,False,False,NaN,{'name': 'Nippon Express NEC Logistics ( Thail...,{'name': 'ภูริวัจน์ (โต้ง)'},{'title': 'Nippon Express NEC Logistics - Toon'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
498,504,11611129,265.0,242.0,NaN,None,Toon 11/01/21 :Called&nbsp; <div><ul><li>Custo...,2021-01-11 07:51:34,2021-01-11 07:51:34,True,False,False,False,False,NaN,None,{'name': 'Phupha ( หมิง )'},{'title': 'Ming - Toon'},"{'email': 'presale@codium.co', 'name': 'Sales ..."


In [41]:
df_note[df_note['deal_id'] == 223]

,id,user_id,deal_id,person_id,org_id,lead_id,content,add_time,update_time,active_flag,pinned_to_deal_flag,pinned_to_person_flag,pinned_to_organization_flag,pinned_to_lead_flag,last_update_user_id,organization,person,deal,user
354,358,11611129,223.0,200.0,172.0,None,CDM next step :&nbsp; <div><ul><li>send on pre...,2020-12-03 08:39:24,2020-12-03 08:39:24,True,False,False,False,False,NaN,{'name': 'KTB Securities'},{'name': 'kitiwat'},{'title': 'KTB Securities - Jeff'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
376,380,11611129,223.0,200.0,172.0,None,Followd up.&nbsp;<br><br>Client currently goin...,2020-12-14 08:06:35,2020-12-14 08:06:35,True,False,False,False,False,NaN,{'name': 'KTB Securities'},{'name': 'kitiwat'},{'title': 'KTB Securities - Jeff'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
424,430,11611129,223.0,200.0,172.0,None,Client said they have chosen us <div><br></div...,2021-01-06 06:29:25,2021-01-06 06:29:25,True,False,False,False,False,NaN,{'name': 'KTB Securities'},{'name': 'kitiwat'},{'title': 'KTB Securities - Jeff'},"{'email': 'presale@codium.co', 'name': 'Sales ..."


In [42]:
df_note[df_note['deal_id'] == 225]

,id,user_id,deal_id,person_id,org_id,lead_id,content,add_time,update_time,active_flag,pinned_to_deal_flag,pinned_to_person_flag,pinned_to_organization_flag,pinned_to_lead_flag,last_update_user_id,organization,person,deal,user
363,367,11611129,225.0,202.0,174.0,None,call 08/12/20 - Customer request to call back ...,2020-12-08 10:12:21,2020-12-08 10:12:21,True,False,False,False,False,NaN,{'name': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำก...,{'name': 'ธัญวดี โพธิ์งาม'},{'title': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำ...,"{'email': 'presale@codium.co', 'name': 'Sales ..."
364,368,11611129,225.0,202.0,174.0,None,call 11/12/20 - Customer not accept phone call,2020-12-11 04:28:09,2020-12-11 04:28:09,True,False,False,False,False,NaN,{'name': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำก...,{'name': 'ธัญวดี โพธิ์งาม'},{'title': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำ...,"{'email': 'presale@codium.co', 'name': 'Sales ..."
370,374,11611129,225.0,202.0,174.0,None,call 14/12/20 - Customer request to call back ...,2020-12-14 06:34:25,2020-12-14 06:34:25,True,False,False,False,False,NaN,{'name': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำก...,{'name': 'ธัญวดี โพธิ์งาม'},{'title': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำ...,"{'email': 'presale@codium.co', 'name': 'Sales ..."
379,383,11611129,225.0,202.0,174.0,None,call 15/12/20 = customer not pick up phone call,2020-12-15 07:01:23,2020-12-15 07:01:23,True,False,False,False,False,NaN,{'name': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำก...,{'name': 'ธัญวดี โพธิ์งาม'},{'title': 'บริษัท ตลิ่งชัน ฮอนด้า ออโตโมบิล จำ...,"{'email': 'presale@codium.co', 'name': 'Sales ..."


In [46]:
df_note[df_note['deal_id'] == 40]

,id,user_id,deal_id,person_id,org_id,lead_id,content,add_time,update_time,active_flag,pinned_to_deal_flag,pinned_to_person_flag,pinned_to_organization_flag,pinned_to_lead_flag,last_update_user_id,organization,person,deal,user
60,61,11611129,40.0,35.0,31.0,None,<span><span>- ประชุม 22.05.20&nbsp;<br>- ลูกค้...,2020-07-13 06:50:03,2020-07-13 06:50:03,True,False,False,False,False,NaN,"{'name': 'Bangkok Komatsu Co, LTD'}",{'name': 'รวิวรรณ'},"{'title': 'Bangkok Komatsu Co, LTD deal'}","{'email': 'presale@codium.co', 'name': 'Sales ..."


In [47]:
# sheet = workbook.get_worksheet(9)
# set_with_dataframe(sheet, df_note)

In [71]:
def strip_html(data):
    import re
    p = re.compile(r'<.*?>|&nbsp;')
    return p.sub('', data)

In [72]:
con = df_note[df_note['deal_id'] == 40]['content']
con.values[0]

'<span><span>- ประชุม 22.05.20&nbsp;<br>- ลูกค้า ขอประชุมภายในเพื่อสรุป scope กันก่อน&nbsp;<br>- ตอนนี้ลค มีส่ง form มาให้ประเมินแล้ว ตอนนี้เหลือรอเขา<br>คุยกันภายในอีกที<br></span><span style="font-weight:bold; color:rgb">- รอ follow up 15.06.20</span></span>'

In [74]:
text = strip_html(con.values[0])
text

'- ประชุม 22.05.20- ลูกค้า ขอประชุมภายในเพื่อสรุป scope กันก่อน- ตอนนี้ลค มีส่ง form มาให้ประเมินแล้ว ตอนนี้เหลือรอเขาคุยกันภายในอีกที- รอ follow up 15.06.20'